In [1]:
import numpy as np
import pickle
import cv2
from os import listdir
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.
C:\Users\INFO-DSK-02\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\INFO-DSK-02\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\INFO-DSK-02\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\INFO-DSK-02\Anaconda3\lib\site-packages\tensorflow\py

In [2]:
EPOCHS = 25
INIT_LR = 1e-3
BS = 32
default_image_size = tuple((256, 256))
image_size = 0
directory_root = 'C:/Users/INFO-DSK-02/Desktop/AI_Farming/PlantVillage'
width=256
height=256
depth=3

In [3]:
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None :
            image = cv2.resize(image, default_image_size)   
            return img_to_array(image)
        else :
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None

In [4]:
image_list= []
label_list = []
try:
    print("[INFO] Loading images ...")
    root_dir = listdir(directory_root)
#     print(root_dir)
    for directory in root_dir :
        # remove .DS_Store from list
        if directory == ".DS_Store" :
            root_dir.remove(directory)

    for plant_folder in root_dir :
#         print(plant_folder)
        plant_disease_folder_list = listdir(f"{directory_root}/{plant_folder}")
        
        for disease_folder in plant_disease_folder_list :
#             print(disease_folder)
            # remove .DS_Store from list
            if disease_folder == ".DS_Store" :
                plant_disease_folder_list.remove(disease_folder)

        for plant_disease_folder in plant_disease_folder_list:
#             print(plant_disease_folder)
#             print(f"[INFO] Processing {plant_disease_folder} ...")
            plant_disease_image_list = listdir(f"{directory_root}/{plant_folder}/{plant_disease_folder}/")
                
            for single_plant_disease_image in plant_disease_image_list :
                if single_plant_disease_image == ".DS_Store" :
                    plant_disease_image_list.remove(single_plant_disease_image)

            for image in plant_disease_image_list[:200]:
                print(image)
                image_directory = f"{directory_root}/{plant_folder}/{plant_disease_folder}/{image}"
                if image_directory.endswith(".jpg") == True or image_directory.endswith(".JPG") == True:
                    
                    image_list.append(convert_image_to_array(image_directory))
                    label_list.append(plant_disease_folder)
    print("[INFO] Image loading completed")  
except Exception as e:
    print(f"Error : {e}")

[INFO] Loading images ...
0022d6b7-d47c-4ee2-ae9a-392a53f48647___JR_B.Spot 8964.JPG
006adb74-934f-448f-a14f-62181742127b___JR_B.Spot 3395.JPG
00f2e69a-1e56-412d-8a79-fdce794a17e4___JR_B.Spot 3132.JPG
01613cd0-d3cd-4e96-945c-a312002037bf___JR_B.Spot 3262.JPG
0169b9ac-07b9-4be1-8b85-da94481f05a4___NREC_B.Spot 9169.JPG
018e494e-d2eb-468b-9d02-40219d9f4921___JR_B.Spot 9045.JPG
01940b6d-7dea-4889-a7b8-a35f4e9bba34___NREC_B.Spot 9120.JPG
01dfb88b-cd5a-420c-b163-51f5fe07b74d___JR_B.Spot 9091.JPG
01ebc916-4793-40a3-b5e4-a32687e4fa3d___NREC_B.Spot 9125.JPG
024623ab-be81-4d99-a653-c3be1495ce8e___JR_B.Spot 8889.JPG
02baf62e-11e2-4dde-97fb-e369b57d55d3___JR_B.Spot 8971.JPG
032aa7c4-0a95-4f09-b0bb-1242f4f32d82___NREC_B.Spot 1889.JPG
03bb7042-3fd5-42e1-aa2c-d51d6843704c___JR_B.Spot 8925.JPG
0448a226-3d3a-494d-a026-a92b048689c7___JR_B.Spot 8897.JPG
0495b4d4-66a4-438c-862d-c483f08e4a95___JR_B.Spot 3288.JPG
04d46cfb-9cc8-4083-82af-ca2bb57c8182___NREC_B.Spot 1814.JPG
05287bcb-610b-440f-9337-c4ce98bc3bbe

2fd2f664-70aa-40d6-b770-bb839944ce18___JR_B.Spot 3261.JPG
301b41f3-647f-4bd2-9adb-ef98c8b8c238___JR_B.Spot 3399.JPG
3042800d-559d-4250-bb62-23c346259685___NREC_B.Spot 1980.JPG
30559581-e3ea-4ac4-8e8b-ff8bbc44b014___JR_B.Spot 8832.JPG
307ade58-8297-4ca9-b412-7ab20f837dc3___NREC_B.Spot 1861.JPG
30956fb2-aec3-4e1d-bc10-ead8d1bd9c03___NREC_B.Spot 1835.JPG
311c06b4-36e7-4fc7-840d-2f3f86d28ea5___JR_B.Spot 3350.JPG
313665a9-0f55-4f00-9373-74bf7349e6a6___NREC_B.Spot 1982.JPG
313c8e04-9195-4b11-9f63-72ce704f78f3___NREC_B.Spot 9245.JPG
315668fd-452d-43c1-8a63-e3b4d3d634a7___JR_B.Spot 3097.JPG
31add1a3-eab6-4c9c-946f-d85d89297a0e___NREC_B.Spot 1842.JPG
3233e1db-99e9-4107-8711-c8b28a8bfda1___JR_B.Spot 3165.JPG
32927179-338a-4157-ab64-d5bea77834c1___JR_B.Spot 8933.JPG
3294ec88-cc8d-4e3a-af08-f2f9d7f4a73b___JR_B.Spot 3196.JPG
32bc632f-7dfc-4a3a-b25a-f71fba5b02b1___JR_B.Spot 8967.JPG
00100ffa-095e-4881-aebf-61fe5af7226e___JR_HL 7886.JPG
00208a93-7687-4e8c-b79e-3138687e0f38___JR_HL 7955.JPG
002f87b7-e

1c665f14-938a-4563-8613-031a2cb04453___JR_HL 5815.JPG
1ce2eb3c-b8a7-4839-9728-a380909f9b89___JR_HL 8448.JPG
1ce80571-2358-4616-b04a-14bbdfd2af6b___JR_HL 8340.JPG
1d0430f7-af9b-4e3d-9d4a-47cf676556fe___JR_HL 5938.JPG
1d6e01ff-affd-4867-895d-fb82b6b0dce3___JR_HL 7952.JPG
1d9f7d60-de05-420d-b620-63d0d1670b76___JR_HL 8837.JPG
1dab0ba9-9982-46ec-9ef6-70a1698baf04___JR_HL 8690.JPG
1daf2391-f049-4e01-b229-ffd7c00a1b73___JR_HL 8119.JPG
1dcc5be2-abd6-4fb3-8e75-d75b7a123e45___JR_HL 8763.JPG
1dd1b153-8ded-439f-8c9e-c9970c67e642___JR_HL 8163.JPG
1dd236c3-5ff0-4a03-996f-e832b50dff82___JR_HL 8787.JPG
1df9cbc2-e14f-4cf8-8ce0-fead77aad080___JR_HL 5855.JPG
1e4bbf82-e2dd-48b2-b9fc-024c20bcdc7b___JR_HL 8172.JPG
1e6e9501-4510-4950-abea-36882683aa5c___JR_HL 8435.JPG
1e9de601-8ffb-42cd-b7ab-79b35f87acfc___JR_HL 8732.JPG
1ed45244-4767-4607-a59e-61342e8f4750___JR_HL 8601.JPG
1f12962f-8c78-44dd-a2fc-fac013a76475___JR_HL 8489.JPG
1f66bf0a-3af3-4084-ab73-c1e1b8f3dc46___JR_HL 8824.JPG
1f700218-0a6a-47ad-a7fa-ded7

25703c4f-ec40-4099-b249-a4fd07b07752___RS_Early.B 7040.JPG
2639ad26-c7e9-4bd6-8000-c002f676a4c6___RS_Early.B 7150.JPG
268bb1c3-a714-46af-b6a5-bea66496158c___RS_Early.B 8678.JPG
26d0502d-5fb3-48fa-a86d-d1de92703285___RS_Early.B 6959.JPG
26d0fa09-d463-461b-830c-3e970244cabb___RS_Early.B 7406.JPG
26d2af6a-f52e-457c-8006-fa106ad39fbe___RS_Early.B 8586.JPG
26f15c9c-5cbf-49c8-ac4e-c5b15d03e319___RS_Early.B 9022.JPG
275c3337-11ee-487c-9093-bcd5e310c72c___RS_Early.B 7671.JPG
2767ef91-3383-4872-9804-e0bcd2bc43fc___RS_Early.B 7783.JPG
2771e005-3cb4-4e12-8868-80adca5ce02a___RS_Early.B 8640.JPG
283134dd-8b32-447e-8e82-547d3b69f4d4___RS_Early.B 7494.JPG
28f275f9-4dd9-479a-a005-ed442362840a___RS_Early.B 6790.JPG
292a3f93-96a2-4b0d-879f-581136c57bcf___RS_Early.B 7099.JPG
29386668-721c-4359-99a6-734f1a4b096b___RS_Early.B 8362.JPG
29508e75-1a9d-4838-b929-cf42f8638e83___RS_Early.B 6782.JPG
29922d76-0eda-4e7c-89af-7688c656bfdd___RS_Early.B 8353.JPG
29978e78-7d4a-4fff-a659-52e45e9b96b3___RS_Early.B 7672.J

c9ad5c9a-c9b0-4e07-9e03-9e9edfd58a19___RS_HL 1755.JPG
d1752ed4-a0a7-4075-917d-205c0e785dcf___RS_HL 1911.JPG
d227e4c8-e203-4aa4-b813-25b96baa2869___RS_HL 1831.JPG
d287be3d-cac6-4485-8c31-674fb8b459be___RS_HL 1973.JPG
d325e580-5faa-4aea-927d-2700f52cc343___RS_HL 1872.JPG
d340704d-cc00-4068-bb8a-ae70840b7bfa___RS_HL 1773.JPG
d50a580b-46bd-47f5-a0bf-8535da774104___RS_HL 5408.JPG
dbe4f332-9189-4b76-b30c-f0c9a55e48d3___RS_HL 1841.JPG
dca8ac5c-b925-435a-a5e9-da09fa8938a1___RS_HL 4182.JPG
de9a8669-d437-40f0-aa56-fe2d8276a1e0___RS_HL 1910.JPG
e1b49d1b-165b-4f4e-8a30-735e62bc39c5___RS_HL 1839.JPG
e34e563f-7a4a-4b44-bdf2-b6f96cbcd929___RS_HL 5370.JPG
e51367c1-d582-45e3-acf6-b0e652b43d3a___RS_HL 1873.JPG
e693354a-053c-4093-8642-cc8f5f9eb2cc___RS_HL 1746.JPG
ec6e43d3-512d-4908-99f2-c9e2a0be54be___RS_HL 4202.JPG
ee052892-e935-4fbb-9344-4d23c57c6890___RS_HL 5400.JPG
ef7005dc-1d44-412e-b858-145a2d7a6fa9___RS_HL 1951.JPG
f0036dfd-9f46-4f2f-9001-5973495f2171___RS_HL 1849.JPG
f0fdde26-78b4-46f0-9eb8-7c2a

...

079c26a2-0862-4062-aa4b-53a5af52350f___Com.G_SpM_FL 1497.JPG
07a04ca5-a3f3-43c7-b035-95eccd5ca39e___Com.G_SpM_FL 1401.JPG
081a628a-9293-4aac-bde6-c117085132f6___Com.G_SpM_FL 9037.JPG
08261e43-0fe0-42bd-9c15-3dca406ba0bf___Com.G_SpM_FL 1359.JPG
085cdef7-8892-4873-a0e7-5715082762d0___Com.G_SpM_FL 8650.JPG
08672d79-7f11-4900-9e12-aad35778f463___Com.G_SpM_FL 8753.JPG
087cd727-92d4-4c49-a069-26b3563037e4___Com.G_SpM_FL 9573.JPG
0889ce0b-097f-4824-a46e-7ed9a01a4862___Com.G_SpM_FL 1435.JPG
089621fa-9c68-4401-a964-ee3383a31cd6___Com.G_SpM_FL 8554.JPG
08ddab1e-8a69-47b2-b0ff-6d53e6315eee___Com.G_SpM_FL 1445.JPG
08e9a879-8784-4775-aeef-21fbd62dd08e___Com.G_SpM_FL 8513.JPG
08f0fd0e-d4b1-400d-9df1-2d61d24c95f0___Com.G_SpM_FL 9397.JPG
093aecee-3211-4aad-8f09-45ccdc92eeb6___Com.G_SpM_FL 9040.JPG
09434fa9-0733-4ef5-9cc3-db29d1e3a480___Com.G_SpM_FL 1425.JPG
0956a6ef-18ce-4a82-a9a1-ca41d07ae13f___Com.G_SpM_FL 9371.JPG
09904a06-b32f-4db4-963f-c8dacfff6b9e___Com.G_SpM_FL 8456.JPG
09dce947-bb58-46cc-bd33-

1d1c9133-33c7-476d-a358-8699d5de8e1d___Com.G_SpM_FL 8915.JPG
1d34e045-0cee-4bea-bd14-3a60de7c255e___Com.G_SpM_FL 1312.JPG
1d7350c2-9ca5-42c8-89bd-6c6f94dd6459___Com.G_SpM_FL 8803.JPG
1d7a6c32-da22-42a0-8dca-85b758836bc6___Com.G_SpM_FL 1452.JPG
1d7bff81-b441-4f32-8cc0-85dd50b79cf3___Com.G_SpM_FL 1443.JPG
1da21007-3d29-487f-b839-e835993f20d2___Com.G_SpM_FL 1389.JPG
1dac8b31-ebbc-4ff5-965c-19f58c368e9e___Com.G_SpM_FL 8843.JPG
1dba797c-45b1-4509-8326-753cf2cf7ebe___Com.G_SpM_FL 8515.JPG
1dd207c1-1fbe-454a-b63f-8051bb26aea8___Com.G_SpM_FL 1161.JPG
1dde88ff-2897-4181-834f-e3629e3cd12c___Com.G_SpM_FL 9560.JPG
1e06dafa-f977-4354-992e-3271f34a2344___Com.G_SpM_FL 9601.JPG
1e09bb35-ef0d-4efb-b3eb-320a976a5e13___Com.G_SpM_FL 1529.JPG
1e0e0b1b-bc70-486b-9d85-f6e3ca348516___Com.G_SpM_FL 1285.JPG
1e0febc9-f544-45bb-9a39-69722d288393___Com.G_SpM_FL 1578.JPG
1e2f0007-3bba-4d4c-b533-879a2be8ac04___Com.G_SpM_FL 9603.JPG
1e802d57-3ff5-4527-8c9a-b1b764d6ccc9___Com.G_SpM_FL 1522.JPG
1ea67ad6-cd45-45f3-a5a0-

1ecca1a2-c028-4cf8-8861-6fcb365bffca___Com.G_TgS_FL 9707.JPG
1f2808be-4fd7-4711-b2dd-5dcc765ef22f___Com.G_TgS_FL 8307.JPG
1f315135-8b90-4b79-ac61-c71a06ab141e___Com.G_TgS_FL 8080.JPG
1f778f17-eacb-4d7a-bf34-041955ebff8f___Com.G_TgS_FL 0694.JPG
1fb2795b-4c60-405e-873b-e3dd3fe6563e___Com.G_TgS_FL 0043.JPG
1fb8b2d9-f0ac-4af2-8ac7-9304558a88e8___Com.G_TgS_FL 7998.JPG
1fef7bcc-bdc0-45dc-a8e6-1c9ee855a675___Com.G_TgS_FL 9950.JPG
20515a64-1d49-4e91-b27f-2d150bd68db2___Com.G_TgS_FL 7926.JPG
205f692a-698d-4572-9b11-b293092cd269___Com.G_TgS_FL 8205.JPG
20ac4321-f374-48d4-89c8-ef09f35f1271___Com.G_TgS_FL 8145.JPG
20e074f5-27bb-48ac-8cb6-bb6586f3aeaa___Com.G_TgS_FL 8200.JPG
216986d3-6b5d-47e6-8ae5-876de7aac386___Com.G_TgS_FL 8199.JPG
217b15a1-dcfc-4cf5-85fa-b305f4ecd708___Com.G_TgS_FL 9919.JPG
21979570-bcc3-481e-bde0-ca5e108d3844___Com.G_TgS_FL 8388.JPG
21aa9ddb-0051-41c7-b1e3-316c20c64483___Com.G_TgS_FL 8151.JPG
21c3078a-7741-49f6-87af-394cc005e18f___Com.G_TgS_FL 7891.JPG
21c8de33-5db5-4cd9-8f69-

656459eb-a35c-426b-8ee9-45d4a4076b5b___PSU_CG 2228.JPG
658e96ee-271d-442b-876e-a52970ff540f___PSU_CG 2269.JPG
65d7e8ff-9197-4211-a1b7-850c3a74f740___PSU_CG 2073.JPG
66778ba1-eb89-4597-913a-71c68eb0251e___PSU_CG 2274.JPG
67076009-be77-4b3f-8de7-bc6f13df64f4___PSU_CG 2209.JPG
67662638-1bfa-4ff6-8305-2d8544277ef0___PSU_CG 2222.JPG
67c65196-795d-4df0-bbd8-0c18af1028c1___PSU_CG 2387.JPG
67ccd0d2-9b55-48a7-b093-a616d9c57845___PSU_CG 2307.JPG
67d62cc2-ac5e-4ad7-91a0-c097163df653___PSU_CG 2048.JPG
6804695f-56f8-4ff9-90f8-69ee95097463___PSU_CG 2067.JPG
6881c9a5-04b2-41c3-b29f-1ca04bd73967___PSU_CG 2390.JPG
696758e5-0f10-4599-b596-e9e7566c4791___PSU_CG 2062.JPG
69726f1b-4d1c-4647-aef2-090e57246501___PSU_CG 2131.JPG
69bad8a8-c403-4e4b-bff8-1026169bb582___PSU_CG 2109.JPG
69d3740b-5aba-47e2-ae2f-46541333c3a5___PSU_CG 2420.JPG
6a2c0438-3207-4998-893a-02ddb3a26081___PSU_CG 2294.JPG
6b15d0fc-1089-4f54-955a-0d1ebbf0ab30___PSU_CG 2364.JPG
6b5eed43-832e-4cca-97eb-534b65635d14___PSU_CG 2050.JPG
6c3a09c7-0

077f081a-67f2-48d2-8084-53b912e65f3c___YLCV_GCREC 2935.JPG
07814785-df21-4b4c-9e52-b43131eaaa76___UF.GRC_YLCV_Lab 09327.JPG
078713ce-c5a0-4351-adcb-00c481417d34___YLCV_GCREC 2304.JPG
078b2cb7-33e2-4444-9ced-0d87a28da668___YLCV_NREC 2146.JPG
079971f3-8298-40ba-b8a5-c389784f5118___YLCV_GCREC 2654.JPG
07a69b1e-1b70-44cc-ad96-24d4200be7d3___UF.GRC_YLCV_Lab 02218.JPG
07aa5223-ed3c-48a9-83e7-bd76a110f941___UF.GRC_YLCV_Lab 03119.JPG
07aadcd9-330a-4600-a4d7-a638498c022d___YLCV_GCREC 2291.JPG
07ad6ef7-f12c-4c74-b3d2-60e57e7db652___YLCV_GCREC 2195.JPG
07b76767-2537-474e-8fba-8c8dd29537a0___UF.GRC_YLCV_Lab 09587.JPG
07c0b302-3b90-48bd-8615-b31b2cf87691___UF.GRC_YLCV_Lab 02927.JPG
07cd23b4-48dd-4ae0-be4a-dfe529e3779a___YLCV_NREC 2930.JPG
07d50744-753e-41f2-b637-99f2ff6c186d___UF.GRC_YLCV_Lab 01558.JPG
07d5a888-c3d3-463b-9245-150a8aec1ce1___YLCV_GCREC 5145.JPG
07ee388e-0206-49a8-bc3a-f53406a49be3___YLCV_GCREC 2920.JPG
07f1d416-ee72-4f16-9840-1c20d02c64cc___UF.GRC_YLCV_Lab 01781.JPG
0808b0d2-02be-48

In [5]:
image_size = len(image_list)

In [7]:
image_directory

'C:/Users/INFO-DSK-02/Desktop/AI_Farming/PlantVillage/Dataset/Tomato__Tomato_YellowLeaf__Curl_Virus/09fc89d4-fffc-4a3b-aa4d-26b0ee5610c8___UF.GRC_YLCV_Lab 02107.JPG'

In [ ]:
label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list)
pickle.dump(label_binarizer,open('label_transform.pkl', 'wb'))
n_classes = len(label_binarizer.classes_)

In [ ]:
print("[INFO] Spliting data to train, test")
x_train_img, x_test_img, y_train_img, y_test_img = train_test_split(image_list, label_list, test_size=0.2, random_state = 42) 

In [ ]:
np_image_list = np.array(image_list, dtype=np.float16) / 225.0

In [ ]:
print("[INFO] Spliting data to train, test")
x_train, x_test, y_train, y_test = train_test_split(np_image_list, image_labels, test_size=0.2, random_state = 42) 

In [ ]:
aug = ImageDataGenerator(
    rotation_range=25, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, 
    zoom_range=0.2,horizontal_flip=True, 
    fill_mode="nearest")

In [ ]:
model = Sequential()
inputShape = (height, width, depth)
chanDim = -1
if K.image_data_format() == "channels_first":
    inputShape = (depth, height, width)
    chanDim = 1
model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(n_classes))
model.add(Activation("softmax"))

In [ ]:
model.summary()

In [ ]:
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
# distribution
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])
# train the network
print("[INFO] training network...")

In [ ]:
history = model.fit_generator(
    aug.flow(x_train, y_train, batch_size=BS),
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // BS,
    epochs=EPOCHS, verbose=1
    )

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
#Train and validation accuracy
plt.plot(epochs, acc, 'b', label='Training accurarcy')
plt.plot(epochs, val_acc, 'r', label='Validation accurarcy')
plt.title('Training and Validation accurarcy')
plt.legend()

plt.figure()
#Train and validation loss
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()
plt.show()

In [ ]:
print("[INFO] Calculating model accuracy")
scores = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100}")

In [ ]:
# save the model to disk
print("[INFO] Saving model...")
pickle.dump(model,open('cnn_model.pkl', 'wb'))

In [ ]:
from sklearn.externals import joblib

In [ ]:
loaded_model = joblib.load('cnn_model.pkl')

In [ ]:
print("[INFO] Calculating model accuracy")
scores = loaded_model.evaluate(x_test[0:1], y_test[0:1])
print(f"Test Accuracy: {scores[1]*100}")

In [ ]:
loaded_model.predict_classes(x_test[0:1])

In [ ]:
y_test_img[0:1]

In [ ]:
print("[INFO] Calculating model accuracy")
scores = loaded_model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100}")

In [7]:
import tkinter as tk
root= tk.Tk()
canvas1 = tk.Canvas(root, width = 1200, height = 450)
canvas1.pack()
# with statsmodels
# print_model =loaded_model
# label_model = tk.Label(root, text=print_model, justify = 'center', relief = 'solid', bg='LightSkyBlue1')
# canvas1.create_window(800, 220, window=label_model)
# New_Interest_Rate label and input box
label0 = tk.Label(root, text='Patient Name: ')
canvas1.create_window(100, 80, window=label0)
entry0 = tk.Entry (root) # create 1st entry box
canvas1.create_window(270, 80, window=entry0)
# New_Interest_Rate label and input box
label1 = tk.Label(root, text='Type No. Pregnancies: ')
canvas1.create_window(100, 100, window=label1)
entry1 = tk.Entry (root) # create 1st entry box
canvas1.create_window(270, 100, window=entry1)
# New_Unemployment_Rate label and input box
label2 = tk.Label(root, text=' Type PlasmaGlucose Level: ')
canvas1.create_window(120, 120, window=label2)
entry2 = tk.Entry (root) # create 2nd entry box
canvas1.create_window(270, 120, window=entry2)
# New_Unemployment_Rate label and input box
label3 = tk.Label(root, text=' Type DiastolicBloodPressure Level: ')
canvas1.create_window(120, 140, window=label3)
entry3 = tk.Entry (root) # create 2nd entry box
canvas1.create_window(270, 140, window=entry3)
# New_Unemployment_Rate label and input box
label4 = tk.Label(root, text=' Type TricepsThickness: ')
canvas1.create_window(120, 160, window=label4)
entry4 = tk.Entry (root) # create 2nd entry box
canvas1.create_window(270, 160, window=entry4)
# New_Unemployment_Rate label and input box
label5 = tk.Label(root, text=' Type SerumInsulin: ')
canvas1.create_window(120, 180, window=label5)
entry5 = tk.Entry (root) # create 2nd entry box
canvas1.create_window(270, 180, window=entry5)
# New_Unemployment_Rate label and input box
label6 = tk.Label(root, text=' Type BMI: ')
canvas1.create_window(120, 200, window=label6)
entry6 = tk.Entry (root) # create 2nd entry box
canvas1.create_window(270, 200, window=entry6)
# New_Unemployment_Rate label and input box
label7 = tk.Label(root, text=' Type DiabetesPedigree: ')
canvas1.create_window(120, 220, window=label7)
entry7 = tk.Entry (root) # create 2nd entry box
canvas1.create_window(270, 220, window=entry7)
# New_Unemployment_Rate label and input box
label8 = tk.Label(root, text=' Type Age: ')
canvas1.create_window(120, 240, window=label8)
entry8 = tk.Entry (root) # create 2nd entry box
canvas1.create_window(270, 240, window=entry8)
def values():
   global New_Patient
   New_Patient = str(entry0.get())
   global New_Pregnancies #our 1st input variable
   New_Pregnancies = float(entry1.get())
   global New_PlasmaGlucose #our 2nd input variable
   New_PlasmaGlucose = float(entry2.get())
   global New_DiastolicBloodPressure #our 2nd input variable
   New_DiastolicBloodPressure = float(entry3.get())
   global New_TricepsThickness #our 2nd input variable
   New_TricepsThickness = float(entry4.get())
   global New_SerumInsulin #our 2nd input variable
   New_SerumInsulin = float(entry5.get())
   global New_BMI #our 2nd input variable
   New_BMI = float(entry6.get())
   global New_DiabetesPedigree #our 2nd input variable
   New_DiabetesPedigree = float(entry7.get())
   global New_Age #our 2nd input variable
   New_Age = float(entry8.get())
   Prediction_result  = ('Predicted Diabetics for : {}'.format(New_Patient), final_estimator.predict([[New_Pregnancies,New_PlasmaGlucose,New_DiastolicBloodPressure,New_TricepsThickness,New_SerumInsulin,New_BMI,New_DiabetesPedigree,New_Age]]))
   label_Prediction = tk.Label(root, text= Prediction_result, bg='yellow')
   canvas1.create_window(270, 280, window=label_Prediction)
button1 = tk.Button (root, text='Predict Diabetics',command=values, bg='orange') # button to call the 'values' command above
canvas1.create_window(270, 300, window=button1)
root.mainloop()